# Offline policy training & evaluation


Inspiration from [this turorial](https://vowpalwabbit.org/docs/vowpal_wabbit/python/latest/tutorials/off_policy_evaluation.html). We use an **incrementally trained** VW policy with adf-format data. 

In [7]:
import os
import random
import vowpalwabbit


In [2]:
cb_type = "ips"

if os.path.isfile('candidate_model.vw'):
    os.remove('candidate_model.vw')


Train the model.

In [3]:
vw_train = vowpalwabbit.Workspace(
                                  cb_explore_adf=True,
                                  cb_type=cb_type,
                                  d="data.dat",
                                  f="candidate_model.vw"
                                    )
vw_train.finish()

final_regressor = candidate_model.vw
using no cache
Reading datafile = data.dat
num sources = 1
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
cb_type = ips
Enabled learners: gd, scorer-identity, csoaa_ldf-rank, cb_adf, cb_explore_adf_greedy, shared_feature_merger
Input label = CB
Output pred = ACTION_PROBS
average  since         example        example        current        current  current
loss     last          counter         weight          label        predict features
1.090909 1.090909            1            1.0      0:1:0.083         0:0.09       11
1.090909 1.090909            2            2.0      0:1:0.083         0:0.09       22
0.816288 0.541667            4            4.0      0:1:0.077         0:0.08       12
0.543561 0.270833            8            8.0      0:0:0.083         0:0.09       33
0.624290 0.705019           16           16.0       0:1:0.14         0:0.17       36
0.554632 0.484975           32           32.0      0:1:0.091          0:0.

Testing

In [4]:
vw_play = vowpalwabbit.Workspace(enable_logging=True, i="candidate_model.vw", cb_type=cb_type)

In [13]:
def sample_custom_pmf(pmf):
    total = sum(pmf)
    scale = 1 / total
    pmf = [x * scale for x in pmf]
    draw = random.random()
    sum_prob = 0.0
    for index, prob in enumerate(pmf):
        sum_prob += prob
        if sum_prob > draw:
            return index, prob


sample = vw_play.predict(
'''shared |vaccines covid_stats
|no_action
|icu_stats_form
|vaccine_stats_form
|pharmacy_form
|ill_questionnaire_form
|ways_of_protection
|symptoms
|test_types
|test_cost
|what_to_do_if_positive
|underlying_conditions''')

sample_custom_pmf(sample)

(1, 0.09090909090909091)

Useful sources:
* https://stackoverflow.com/questions/66996884/confusion-with-vowpal-wabbit-contextual-bandit-training-data-formatting
* https://stackoverflow.com/questions/28640837/vowpal-wabbit-how-to-represent-categorical-features/28643968#28643968